In [1]:
import pandas as pd

In [3]:
caminho = rf"C:\Users\Arthur\Downloads\reclamacoes-n1e2-distribuidoras-2024.csv"

In [4]:
chunksize = 1_000_000
lista = []
i = 0

for chunk in pd.read_csv(caminho, encoding='latin1', sep=';', chunksize=chunksize):
    i += 1
    print(f"Lendo partição {i}")
    lista.append(chunk)

df_total = pd.concat(lista)

Lendo partição 1
Lendo partição 2
Lendo partição 3
Lendo partição 4
Lendo partição 5
Lendo partição 6
Lendo partição 7
Lendo partição 8
Lendo partição 9


In [5]:
df_total

,DatGeracaoConjuntoDados,DatReferencia,SigAgente,NumCPFCNPJ,CodMunicipio,NomMunicipio,SigUF,SigRegiao,NomClassificacaoAgente,CodTipoReclamacao,NomCanalReclamacao,DescReclamacao,QtdReclamacoesRecebidas,QtdReclamacoesImprocedentes,QtdReclamacoesProcedentes,NumPrazoMedioSolucao,DscFormaContato,NumOuvPrazoMedioSolucaoImproc,NumSacPrazoMedioSolucaoImproc
0,2025-08-09,2024-02-29,Amazonas Energia,2341467000120,1301159,Careiro da Várzea,AM,N,Concessionária,1020101,Nível 1,Solicitação não atendida ou atrasada,1,0,0,",00",Telefônico,NaN,NaN
1,2025-08-09,2024-02-29,Amazonas Energia,2341467000120,1303569,Rio Preto da Eva,AM,N,Concessionária,1020101,Nível 1,Solicitação não atendida ou atrasada,1,0,0,",00",Telefônico,NaN,NaN
2,2025-08-09,2024-02-29,Amazonas Energia,2341467000120,1302603,Manaus,AM,N,Concessionária,1020101,Nível 1,Solicitação não atendida ou atrasada,3,0,0,",00",Telefônico,NaN,NaN
3,2025-08-09,2024-02-29,Amazonas Energia,2341467000120,1300300,Autazes,AM,N,Concessionária,1020101,Nível 1,Solicitação não atendida ou atrasada,1,0,0,",00",Outros,NaN,NaN
4,2025-08-09,2024-02-29,Amazonas Energia,2341467000120,1303403,Parintins,AM,N,Concessionária,1020101,Nível 1,Solicitação não atendida ou atrasada,1,0,1,"6,04",Telefônico,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8021947,2025-08-09,2024-12-31,Sulgipe,13255658000196,2803005,Itabaianinha,SE,NE,Concessionária,10106,Nível 2,Tarifas / Fatura / Faturamento / Cobrança,2,0,0,",00",Telefônico,NaN,NaN
8021948,2025-08-09,2024-12-31,Sulgipe,13255658000196,2802809,Indiaroba,SE,NE,Concessionária,10106,Nível 2,Tarifas / Fatura / Faturamento / Cobrança,1,0,0,",00",Telefônico,NaN,NaN
8021949,2025-08-09,2024-12-31,Sulgipe,13255658000196,2807600,Umbaúba,SE,NE,Concessionária,10106,Nível 2,Tarifas / Fatura / Faturamento / Cobrança,2,0,0,",00",Telefônico,NaN,NaN
8021950,2025-08-09,2024-12-31,Sulgipe,13255658000196,2807402,Tobias Barreto,SE,NE,Concessionária,10106,Nível 2,Tarifas / Fatura / Faturamento / Cobrança,1,0,0,",00",Telefônico,NaN,NaN


In [8]:
df_total.columns

Index(['DatGeracaoConjuntoDados', 'DatReferencia', 'SigAgente', 'NumCPFCNPJ',
       'CodMunicipio', 'NomMunicipio', 'SigUF', 'SigRegiao',
       'NomClassificacaoAgente', 'CodTipoReclamacao', 'NomCanalReclamacao',
       'DescReclamacao', 'QtdReclamacoesRecebidas',
       'QtdReclamacoesImprocedentes', 'QtdReclamacoesProcedentes',
       'NumPrazoMedioSolucao', 'DscFormaContato',
       'NumOuvPrazoMedioSolucaoImproc', 'NumSacPrazoMedioSolucaoImproc'],
      dtype='object')

In [6]:
df_total.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8021952 entries, 0 to 8021951
Data columns (total 19 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   DatGeracaoConjuntoDados        object 
 1   DatReferencia                  object 
 2   SigAgente                      object 
 3   NumCPFCNPJ                     int64  
 4   CodMunicipio                   int64  
 5   NomMunicipio                   object 
 6   SigUF                          object 
 7   SigRegiao                      object 
 8   NomClassificacaoAgente         object 
 9   CodTipoReclamacao              int64  
 10  NomCanalReclamacao             object 
 11  DescReclamacao                 object 
 12  QtdReclamacoesRecebidas        int64  
 13  QtdReclamacoesImprocedentes    int64  
 14  QtdReclamacoesProcedentes      int64  
 15  NumPrazoMedioSolucao           object 
 16  DscFormaContato                object 
 17  NumOuvPrazoMedioSolucaoImproc  float64
 18  Nu